# LLMS

## Set up your `TOGETHER_API_KEY`

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

## Install Packages to Call the LLM

For this part we set up the LLM.

In [ ]:
!pip3 install openai

In [ ]:
import openai
import textwrap

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []

    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines

    return '\n'.join(wrapped_lines)


def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(prompt)
    print("")
    print("Response: ")
    print(response)

def get_completion(prompt, temperature=0.0):
    client = openai.OpenAI(api_key=os.environ.get("TOGETHER_API_KEY"),
                           base_url="https://api.together.xyz/v1")

    chat_completion = client.chat.completions.create(
                           model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",
                           messages=[{"role": "user", "content": prompt}],
                           temperature=temperature,
                           max_tokens=1024)

    response = chat_completion.choices[0].message.content
    response = response.lstrip()
    wrapped_response = wrap_text(response)
    return wrapped_response

## Calling an LLM with `get_completion`

First set up a prompt we can use:

In [ ]:
prompt = "Why is the sky blue? In 50 words or less."

Now we call the`get_completion` function. This function takes our prompt
as argument and sends it off to the LLM. The LLM returns a response.
This function returns that response.

In [ ]:
response = get_completion(prompt)
print(response)

Sometimes its nice to print out the prompt and the response when we are
working like this. We can use the function `print_prompt_and_response`
and pass it the prompt and the response we got from the `get_completion`
function.

In [ ]:
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

## Calling an LLM with the `temperature`

Temperature is one of the settings that most LLMs have. You can set the
temperature from 0.0 up to some maximum (like 1.0 or 1.2). Sometimes you
can go higher.

Roughly

1.  Low temperature (eg 0.2-0.5)
    -   Generates more predictable and conservative text
2.  Medium temperature (eg. 0.6-0.8)
    -   Balances between creativity and coherence
3.  High temperature (eg. 0.9-1.2)
    -   Generates more creative diverse, and unpredictable text

First lets set up a prompt to call several times with different
temperatures.

In [ ]:
prompt = "Why is the sky blue? In 50 words or less"

### Calling with `temperature=0.0`

`temperature` is parameter you can pass to the `get_completion`
function. You see this in the code below.

Here we call the LLM 3 times with temperature 0.0

In [ ]:
response = get_completion(prompt, temperature = 0.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.0)
print(response)
print("------")

### Calling with `temperature=0.4`

Here we call the LLM 3 times with temperature 0.4

In [ ]:
response = get_completion(prompt, temperature = 0.4)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.4)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.4)
print(response)
print("------")

### Calling with `temperature=1.0`

In [ ]:
response = get_completion(prompt, temperature = 1.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 1.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 1.0)
print(response)
print("------")

## Calling the LLM using a template

Remember the template for having the LLM give us the name of a company
that makes a particular thing?

In [ ]:
template = "Recommend a name for a company that makes {what}"

Lets set that up and call it with `what="cars"`:

In [ ]:
prompt=template.format(what="cars")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

## Change the template variable `{what}`

Let’s call the LLM using `what="books"` as the template variable.

In [ ]:
prompt=template.format(what="books")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

## Add A Template Variable `{num}`

In [ ]:
template = "Recommend {num} names for a company that makes {what}"

Lets call it with `num=3` and `what="cars"`:

In [ ]:
prompt=template.format(num=3, what="cars")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

### Add a Template Variable `{format}`

Lets see if we can introduce some formatting to the output:

In [ ]:
template ="""Recommend {num} names for a company that makes {what}.
Give your answer in {format}"""

Call the LLM:

In [ ]:
prompt=template.format(num=3, what="cars", format="A bulleted list using upper case")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)